<br>
@Author: Ayush Prajapati<br>
@Date: 14-10-2024<br>
@Last Modified by: Ayush Prajapati<br>
@Last Modified time: 14-10-2024<br>
@Title: Python program to perform Gen AI tasks Summarizing and Transforming using Gemini API<br>

In [1]:
%pip install google-generativeai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


<br> 

**Importing Libraries**

In [34]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import csv

<br> 

**Loading Env variable**

In [35]:
load_dotenv()

True

<br> 

**Config Gemini AI**

In [36]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

In [37]:
generation_config = {
  "temperature": 0.9,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

In [40]:
def read_emails(file_path, delimiter="---END OF EMAIL---"):
    """
    Description: 
        This function reads the email from the text file
    Parameters:
        file_path: Path of txt file
        delimiter: delimiter
    Return:
        list: list of emails
    """
    with open(file_path, 'r') as file:
        emails = file.read().split(delimiter)
    return [email.strip() for email in emails if email.strip()]  # Return non-empty emails

In [41]:
def extract_email_info(email):
    """
    Description: 
        This Function extracts sender, receiver, and body from email
    Parameters:
        email
    Return:
        sender, receiver, and body (str)
    """
    lines = email.split("\n")
    sender = receiver = body = ""
    
    for line in lines:
        if line.startswith("From:"):
            sender = line.split(":")[1].strip()
        elif line.startswith("To:"):
            receiver = line.split(":")[1].strip()
        elif not line.startswith("Subject:") and not line.startswith("From:") and not line.startswith("To:"):
            body += line.strip() + " "
    
    return sender, receiver, body.strip()

<br> 

**Summarizing Task**

In [42]:
def summarize_email(body, chat_session):
    """
    Description: 
        This Function summarizes email body using the Gemini model
    Parameters:
        body: body of email
        chat_session: gemini ai chat session
    Return:
        response(str): summarized response
    """
    response = chat_session.send_message(f"Summarize the following email: {body}")
    return response.text

<br> 

**Translation Task**

In [47]:
def translate_email(text, chat_session):
    """
    Description: 
        This Function translates text using the Gemini model
    Parameters:
        body: body of email
        chat_session: gemini ai chat session
    Return:
        response(str): summarized response
    """
    response = chat_session.send_message(f"Translate the following text to German: {text}")
    return response.text

In [43]:
def save_to_csv(data, csv_file):
    """
    Description: 
        This function save the response in csv file
    Parameters:
        data: data to be saved in csv
        csv_file: csv file name/path
    """
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Sender", "Receiver", "Summary (EN)", "Summary (DE)"])
        writer.writerows(data)

In [1]:
def process_emails(file_path, csv_file, chat_session, delimiter="---END OF EMAIL---"):
    """
    Description: 
        This function processes the email 
    Parameters:
        file_path: file of email texts
        csv_file: name/path of csv file
        chat_session: gemini ai chat session
    Return:
        response(str): summarized response
    """
    emails = read_emails(file_path, delimiter)
    data = []

    for email in emails:
        sender, receiver, body = extract_email_info(email)
        summary_en = summarize_email(body, chat_session)
        summary_de = translate_email(summary_en, chat_session)
        data.append([sender, receiver, summary_en, summary_de])
    
    save_to_csv(data, csv_file)


In [46]:
def main():
    chat_session = model.start_chat(
        history=[]
    )
    process_emails('mails.txt', 'processed_emails.csv', chat_session)


if __name__ == '__main__':
    main()